# 1)  Imports & global configuration

In [1]:
# Globals – shared by every other cell
from __future__ import annotations

import io, logging, os, time, zipfile
from pathlib import Path
from typing  import Dict, List, Optional

import psycopg2
from dotenv import load_dotenv
from postgre_connector import Postgres  # import the Postgres helper

load_dotenv()                            # .env → os.environ

# ── Project folders ────────────────────────────────────────────────
# Automatically locate the main repository root
# The loop walks upward until both 'airflow/dags' and 'streamlit_app'
# directories are found — this ensures we’re inside the real project,
# not inside a nested notebook folder.
REPO_ROOT = Path.cwd().resolve()
while REPO_ROOT != REPO_ROOT.parent:
    has_airflow = (REPO_ROOT / "airflow" / "dags").is_dir()
    has_streamlit = (REPO_ROOT / "streamlit_app").is_dir()
    if has_airflow and has_streamlit:
        break
    REPO_ROOT = REPO_ROOT.parent

# Define dataset directories used by Airflow
BASE_DIR       = REPO_ROOT / "airflow" / "datasets"
SOURCE_ZIP_DIR = BASE_DIR / "public-zips"
CSV_DIR        = BASE_DIR / "public-data"

# Ensure the dataset folders exist before loading or writing files
for p in (SOURCE_ZIP_DIR, CSV_DIR):
    p.mkdir(parents=True, exist_ok=True)

# ── logging – INFO by default ──────────────────────────────────────
import logging
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s  %(levelname)-8s  %(name)s ➜ %(message)s",
    datefmt="%H:%M:%S",
)
log = logging.getLogger("nb-cnpj-ingest")
log.info("REPO_ROOT      → %s", REPO_ROOT)
log.info("SOURCE_ZIP_DIR → %s", SOURCE_ZIP_DIR)
log.info("CSV_DIR        → %s", CSV_DIR)


# ── column layouts (Portuguese names) ──────────────────
LAYOUTS: Dict[str, List[str]] = {
    "header": [
        "tipo_do_registro", "filler", "nome_do_arquivo", "data_de_gravacao",
        "numero_da_remessa", "filler2", "fim_de_registro"
    ],
    "cnae": ["codigo", "descricao"],
    "empresas": [
        "cnpj_basico", "razao_social", "natureza_juridica",
        "qualificacao_responsavel", "capital_social_str", "porte_empresa",
        "ente_federativo_responsavel"
    ],
    "estabelecimento": [
        "cnpj_basico", "cnpj_ordem", "cnpj_dv", "matriz_filial", 
        "nome_fantasia", "situacao_cadastral", "data_situacao_cadastral",
        "motivo_situacao_cadastral", "nome_cidade_exterior", "pais",
        "data_inicio_atividades", "cnae_fiscal", "cnae_fiscal_secundaria",
        "tipo_logradouro", "logradouro", "numero", "complemento", 
        "bairro", "cep", "uf", "municipio", "ddd1", "telefone1", 
        "ddd2", "telefone2", "ddd_fax", "fax", "correio_eletronico", 
        "situacao_especial", "data_situacao_especial"
    ],
    "motivo": ["codigo", "descricao"],
    "municipio": ["codigo", "descricao"],
    "natureza_juridica": ["codigo", "descricao"],
    "pais": ["codigo", "descricao"],
    "qualificacao_socio": ["codigo", "descricao"],
    "simples": [
        "cnpj_basico", "opcao_simples", "data_opcao_simples",
        "data_exclusao_simples", "opcao_mei", "data_opcao_mei",
        "data_exclusao_mei"
    ],
    "socios_original": [
        "cnpj_basico", "identificador_de_socio", "nome_socio", 
        "cnpj_cpf_socio", "qualificacao_socio", "data_entrada_sociedade", 
        "pais", "representante_legal", "nome_representante", 
        "qualificacao_representante_legal", "faixa_etaria"
    ],
    "cnaes_secundarias": [
        "tipo_do_registro", "indicador_full_diario", "tipo_de_atualizacao",
        "cnpj", "cnae_secundaria", "filler", "fim_de_registro"
    ],
    "trailler": [
        "tipo_do_registro", "filler", "total_de_registros_t1", "total_de_registros_t2",
        "total_de_registros_t3", "total_de_registros", "filler2", "fim_de_registro"
    ]
}


21:58:42  INFO      nb-cnpj-ingest ➜ REPO_ROOT      → /Users/viniciuslamb/Documents/portifolio_cnpj
21:58:42  INFO      nb-cnpj-ingest ➜ SOURCE_ZIP_DIR → /Users/viniciuslamb/Documents/portifolio_cnpj/airflow/datasets/public-zips
21:58:42  INFO      nb-cnpj-ingest ➜ CSV_DIR        → /Users/viniciuslamb/Documents/portifolio_cnpj/airflow/datasets/public-data


# 2)  ZIP → CSV  (pure extraction, no transformation)

In [2]:
# ------------------------------------------------------------------ #
# 1·a)  ZIP → CSV extractor (pure “dump”, zero transformation)
# ------------------------------------------------------------------ #
import time
import zipfile
from pathlib import Path
from typing import Optional

def extract_zip_to_csv(limit: Optional[int] = None) -> None:
    """
    Decompress every ``*.zip`` found under :pydata:`SOURCE_ZIP_DIR`
    and write the *verbatim* contents to :pydata:`CSV_DIR`.

    One archive may contain multiple files – each gets an output name
    like:

        {zip_stem}_{inner_filename}.csv

    Examples
    --------
    >>> extract_zip_to_csv()        # process everything
    >>> extract_zip_to_csv(limit=2) # process only the first 2 ZIPs

    Parameters
    ----------
    limit : int | None, optional
        Maximum number of ZIP archives to process.  Passing *None*
        (default) means “process them all”.

    Notes
    -----
    * **No** data transformation happens here – we copy the bytes 1-for-1.
    * Already-extracted files are skipped (idempotent behaviour).
    * Detailed logging shows progress and per-file timing.
    """

    # ----------------------------------------------------------------
    # 0)  Gather and optionally truncate the list of ZIP archives
    # ----------------------------------------------------------------
    zip_paths = sorted(
        p for p in SOURCE_ZIP_DIR.iterdir()
        if p.suffix.lower() == ".zip"
    )

    if not zip_paths:
        log.warning("⚠️  No ZIP archives found in %s – nothing to extract.", SOURCE_ZIP_DIR)
        return

    if limit is not None:
        zip_paths = zip_paths[:limit]                    # truncate for quick tests

    total_archives = len(zip_paths)
    processed      = 0
    t_global_start = time.perf_counter()

    log.info("🔧  ZIP ➜ CSV extraction started (archives=%s)", total_archives)

    # ----------------------------------------------------------------
    # 1)  Walk each .zip and dump everything inside
    # ----------------------------------------------------------------
    for idx, zip_path in enumerate(zip_paths, start=1):
        t_start = time.perf_counter()
        log.info("→ [%s/%s]  %s", idx, total_archives, zip_path.name)

        try:
            with zipfile.ZipFile(zip_path) as zf:
                for inner_name in zf.namelist():
                    # Build the output filename:  {zip_stem}_{inner_basename}.csv
                    out_path = CSV_DIR / f"{zip_path.stem}_{Path(inner_name).name}.csv"

                    # Skip if we already have it – idempotency is ❤️
                    if out_path.exists():
                        log.debug("   • skip (exists)  %s", out_path.name)
                        continue

                    # Copy the bytes verbatim (no decoding here)
                    with zf.open(inner_name) as src, out_path.open("wb") as dst:
                        dst.write(src.read())

                    log.debug("   • extracted        %s", out_path.name)

            processed += 1
            log.info("   ✔  done in %.2fs", time.perf_counter() - t_start)

        except zipfile.BadZipFile as err:
            # Corrupted archive – log and carry on
            log.error("❌  Bad ZIP %s – %s", zip_path.name, err)

    # ----------------------------------------------------------------
    # 2)  Global summary
    # ----------------------------------------------------------------
    total_time = time.perf_counter() - t_global_start
    log.info("🏁  Extraction finished: %s / %s archive(s) processed in %.2fs",
             processed, total_archives, total_time)


# 3)  Identify which layout a CSV belongs to

In [3]:
# ------------------------------------------------------------------ #
# 3)  Identify which layout a CSV belongs to
# ------------------------------------------------------------------ #
from pathlib import Path
from typing import Optional

def layout_key(file_name: str) -> Optional[str]:
    """
    Inspect *file_name* and return the matching entry inside
    the global :pydata:`LAYOUTS` dict.

    The match is intentionally **simple**:  
    we look for each layout keyword (``"empresas"``, ``"cnae"``,
    ``"estabelecimento"`` …) as a *substring* of the file name
    – case-insensitive.  The **first** hit wins.

    Parameters
    ----------
    file_name : str
        Any file name or full path (only the final component is used).

    Returns
    -------
    str | None
        * The layout key (e.g. ``"empresas"``) when a match is found;
        * **None** when no keyword matches.
    """
    # Normalize: keep just the final component and lower-case it
    fname_low = Path(file_name).name.lower()

    # 1) Direct match against our canonical keys in LAYOUTS
    #    (layout keys already reflect business names we use in tables)
    for key in LAYOUTS:                  # LAYOUTS is defined at module top
        if key in fname_low:
            return key                   # ← success

    # 2) Common aliases found in Receita filenames (abbreviations)
    #    We keep this list local to the function for clarity/encapsulation.
    #    Order matters only if two aliases could match the same file name.
    aliases = {
        # canonical_key              # examples of filename substrings
        "natureza_juridica": ["natju", "natureza"],
        "qualificacao_socio": ["quals", "qualific"],
        "socios_original":    ["socio", "socios"],
        "estabelecimento":    ["estabelec", "estab"],
        "empresas":           ["empre", "empresa", "empresas"],
        "cnae":               ["cnae"],
        "motivo":             ["motic", "motivo"],
        "municipio":          ["munic", "municipio"],
        "pais":               ["pais"],
        "cnaes_secundarias":  ["secundaria", "secundarias"],
        "trailler":           ["trailler", "trailer"],
        "header":             ["header", "cabecalho"],
    }

    for canonical, subs in aliases.items():
        if any(sub in fname_low for sub in subs):
            return canonical

    # Fallback – unknown file
    return None


# 4)  Create *typed* temp tables (one per CNPJ layout)

In [4]:
# ------------------------------------------------------------------ #
# 4)  Create *typed* temp tables (one per CNPJ layout)
# ------------------------------------------------------------------ #
def create_temp_tables() -> None:
    """
    Build/rebuild every **temp_*** table in PostgreSQL using the
    *true* data-types published by Receita Federal.  
    All DDLs start with ``DROP TABLE IF EXISTS`` ➜ the helper is
    **idempotent** and can run safely on every DAG execution.
    """
    log.info("🔧  (Re)creating typed temp tables …")

    ddl_statements = [
        # ------------------------------------------------------------------
        # cnae / industry_classification
        # ------------------------------------------------------------------
        """
        DROP TABLE IF EXISTS temp_cnae CASCADE;
        CREATE TABLE temp_cnae (
            codigo     CHAR(7),
            descricao  TEXT     NOT NULL
        );
        """,
        # ------------------------------------------------------------------
        # empresas / companies
        # ------------------------------------------------------------------
        """
        DROP TABLE IF EXISTS temp_empresas CASCADE;
        CREATE TABLE temp_empresas (
            cnpj_basico                 CHAR(8),
            razao_social                TEXT       NOT NULL,
            natureza_juridica           CHAR(4)    NOT NULL,
            qualificacao_responsavel    CHAR(2)    NOT NULL,
            capital_social              TEXT,
            porte_empresa               CHAR(2),
            ente_federativo_responsavel TEXT
        );
        """,

        # ------------------------------------------------------------------
        # estabelecimento / establishments
        # ------------------------------------------------------------------
"""
        DROP TABLE IF EXISTS temp_estabelecimento CASCADE;
        CREATE TABLE temp_estabelecimento (
            cnpj_basico               CHAR(8),
            cnpj_ordem                CHAR(4),
            cnpj_dv                   CHAR(2),
            matriz_filial             CHAR(1),
            nome_fantasia             TEXT,
            situacao_cadastral        CHAR(2),
            data_situacao_cadastral   TEXT,
            motivo_situacao_cadastral CHAR(2),
            nome_cidade_exterior      TEXT,
            pais                      CHAR(3),
            data_inicio_atividades    TEXT,
            cnae_fiscal               CHAR(7),
            cnae_fiscal_secundaria    TEXT,
            tipo_logradouro           TEXT,
            logradouro                TEXT,
            numero                    VARCHAR(10),
            complemento               TEXT,
            bairro                    TEXT,
            cep                       CHAR(8),
            uf                        CHAR(2),
            municipio                 CHAR(4),
            ddd1                      CHAR(4),
            telefone1                 CHAR(8),
            ddd2                      CHAR(4),
            telefone2                 CHAR(8),
            ddd_fax                   CHAR(4),
            fax                       CHAR(8),
            correio_eletronico        TEXT,
            situacao_especial         TEXT,
            data_situacao_especial    TEXT
        );
        """,

        # ------------------------------------------------------------------
        # motivo / status_reason
        # ------------------------------------------------------------------
        """
        DROP TABLE IF EXISTS temp_motivo CASCADE;
        CREATE TABLE temp_motivo (
            codigo    CHAR(2),
            descricao TEXT    NOT NULL
        );
        """,

        # ------------------------------------------------------------------
        # municipio / municipalities
        # ------------------------------------------------------------------
        """
        DROP TABLE IF EXISTS temp_municipio CASCADE;
        CREATE TABLE temp_municipio (
            codigo    CHAR(4),
            descricao TEXT NOT NULL
        );
        """,

        # ------------------------------------------------------------------
        # natureza_juridica / legal_nature
        # ------------------------------------------------------------------
        """
        DROP TABLE IF EXISTS temp_natureza_juridica CASCADE;
        CREATE TABLE temp_natureza_juridica (
            codigo    CHAR(4) PRIMARY KEY,
            descricao TEXT NOT NULL
        );
        """,

        # ------------------------------------------------------------------
        # pais / countries
        # ------------------------------------------------------------------
        """
        DROP TABLE IF EXISTS temp_pais CASCADE;
        CREATE TABLE temp_pais (
            codigo    CHAR(3) PRIMARY KEY,
            descricao TEXT NOT NULL
        );
        """,

        # ------------------------------------------------------------------
        # qualificacao_socio / partner_qualification
        # ------------------------------------------------------------------
        """
        DROP TABLE IF EXISTS temp_qualificacao_socio CASCADE;
        CREATE TABLE temp_qualificacao_socio (
            codigo    CHAR(2) PRIMARY KEY,
            descricao TEXT NOT NULL
        );
        """,

        # ------------------------------------------------------------------
        # simples / simple_tax_regime
        # ------------------------------------------------------------------
        """
        DROP TABLE IF EXISTS temp_simples CASCADE;
        CREATE TABLE temp_simples (
            cnpj_basico            CHAR(8),
            opcao_simples          CHAR(1),
            data_opcao_simples     DATE,
            data_exclusao_simples  DATE,
            opcao_mei              CHAR(1),
            data_opcao_mei         DATE,
            data_exclusao_mei      DATE
        );
        """,

        # ------------------------------------------------------------------
        # socios_original / original_partners
        # ------------------------------------------------------------------
        """
        DROP TABLE IF EXISTS temp_socios_original CASCADE;
        CREATE TABLE temp_socios_original (
            cnpj_basico                      CHAR(8),
            identificador_de_socio           CHAR(1),
            nome_socio                       TEXT,
            cnpj_cpf_socio                   CHAR(14),
            qualificacao_socio               CHAR(2),
            data_entrada_sociedade           DATE,
            pais                             CHAR(3),
            representante_legal              CHAR(11),
            nome_representante               TEXT,
            qualificacao_representante_legal CHAR(2),
            faixa_etaria                     CHAR(1)
        );
        """
    ]

    pg = Postgres()
    for ddl in ddl_statements:
        log.debug("Executing DDL:\n%s", ddl.strip())
        pg.execute_sql(ddl)

    log.info("✅  Temp tables successfully (re)created.")


# 5)  Low-level I/O · robust COPY with encoding detection (UTF-8/LATIN-1) and disk-backed NUL-strip fallback

In [5]:
def _has_nul_bytes(p: Path, probe_bytes: int = 64 * 1024) -> bool:
    """
    Quickly probe for rogue NUL bytes in a CSV file.

    Rationale
    ---------
    PostgreSQL COPY rejects NUL bytes. Scanning the entire file would be
    wasteful, so we heuristically check only the first *probe_bytes*.
    In practice, if NUL appears in the head, it tends to appear elsewhere.

    Parameters
    ----------
    p : Path
        CSV file path to probe.
    probe_bytes : int, default 64 KiB
        Number of bytes to read from the head of the file.

    Returns
    -------
    bool
        True  -> at least one NUL ('\\x00') found in the probed slice.
        False -> no NUL found in the slice (not a formal proof of absence).
    """
    with p.open("rb") as fh:
        return b"\x00" in fh.read(probe_bytes)

def _detect_csv_encoding(p: Path, probe_bytes: int = 128 * 1024) -> str:
    """
    Decide the best text encoding for a CSV file between **UTF-8** and **Latin-1**.

    Strategy
    --------
    • Read only the first *probe_bytes* to decide quickly (no full scan).
    • If there is a UTF-8 BOM (EF BB BF) → return **'utf-8-sig'**.
    • Else, try to decode that slice as UTF-8:
        – If it succeeds → return **'utf-8'**.
        – If it raises UnicodeDecodeError → fall back to **'latin-1'**.
      (Latin-1 never raises on arbitrary bytes, so order matters.)

    Parameters
    ----------
    p : Path
        Path to the CSV file on disk.
    probe_bytes : int, default 128 KiB
        Number of bytes to sample from the file head.

    Returns
    -------
    str
        One of: **'utf-8-sig'**, **'utf-8'**, or **'latin-1'**.
    """
    with p.open("rb") as fh:
        head = fh.read(probe_bytes)

    # BOM present → UTF-8 with signature
    if head.startswith(b"\xEF\xBB\xBF"):
        return "utf-8-sig"

    # Try a plain UTF-8 decode on the sample
    try:
        head.decode("utf-8")
        return "utf-8"
    except UnicodeDecodeError:
        return "latin-1"


def _copy_csv_to_pg(csv_path: Path, target_table: str) -> None:
    """
    Stream **one** CSV into PostgreSQL via `COPY … FROM STDIN` (CSV mode),
    with **per-file encoding detection** and a **NUL-strip** disk-backed fallback.

    Robustness & memory-safety
    --------------------------
    1) **Encoding detection per file** (UTF-8 or Latin-1) to avoid mojibake
       in the dimension tables (e.g., acentos).
    2) **Fast path**: stream the file with the detected encoding directly
       into `COPY` (low memory, no materialization).
    3) **NUL handling**: if COPY fails and the error/probe suggests NUL bytes,
       write a **temporary cleaned file** stripping only `\\x00` line-by-line,
       then COPY from it. Avoids building giant in-memory buffers.
    4) **Per-table CSV tweaks**: use `FORCE_NULL` on known date-like columns
       so that `""` becomes SQL NULL *before* casting happens downstream.

    Receita CNPJ CSV specifics
    --------------------------
    • Delimiter: `;`
    • No header row (HEADER false)
    • Empty strings `""` → NULL (NULL '')
    • Some files are UTF-8; others Latin-1 (ISO-8859-1)

    Parameters
    ----------
    csv_path : Path
        Absolute path to the CSV file to ingest.
    target_table : str
        Destination temp table name (e.g., `'temp_empresas'`).

    Raises
    ------
    Exception
        Any non-NUL related COPY error; or a failure that persists even
        after the NUL-strip fallback.
    """
    # ── Per-table CSV options kept local (encapsulation) ────────────────────
    _COPY_PER_TABLE_OPTS = {
        "temp_estabelecimento": (
            "FORCE_NULL (data_situacao_cadastral, data_inicio_atividades, data_situacao_especial)"
        ),
        "temp_simples": (
            "FORCE_NULL (data_opcao_simples, data_exclusao_simples, data_opcao_mei, data_exclusao_mei)"
        ),
        "temp_socios_original": "FORCE_NULL (data_entrada_sociedade)",
        # add others if new date-like TEXT columns appear
    }

    # 0) Detect per-file encoding and map to Postgres client_encoding
    file_encoding = _detect_csv_encoding(csv_path)
    client_enc   = "UTF8" if file_encoding.startswith("utf-") else "LATIN1"

    pg   = Postgres()                     # thin wrapper (env via dotenv)
    conn = pg.connect_postgres()
    cur  = conn.cursor()

    # Session knobs for predictable COPY STDIN behavior
    cur.execute(f"SET client_encoding TO '{client_enc}';")   # COPY STDIN uses session encoding
    cur.execute("SET datestyle TO ISO, YMD;")                # predictable date parsing downstream

    base_opts  = "FORMAT csv, DELIMITER ';', NULL '', HEADER false"
    extra_opts = _COPY_PER_TABLE_OPTS.get(target_table, "")
    copy_sql   = f"COPY {target_table} FROM STDIN WITH ({base_opts}{', ' + extra_opts if extra_opts else ''})"

    try:
        # ── 1) Fast path — raw stream in detected text encoding ────────────
        with csv_path.open("r", encoding=file_encoding, errors="replace", newline="") as fh:
            cur.copy_expert(copy_sql, fh)
        conn.commit()
        log.debug("✓ raw COPY (%s)  %s → %s", file_encoding, csv_path.name, target_table)
        return

    except Exception as exc_raw:
        # ── 2) Decide whether we should attempt the NUL-cleaning fallback ──
        conn.rollback()
        msg = str(exc_raw).lower()
        nul_suspect = ("nul" in msg) or ("invalid byte" in msg) or _has_nul_bytes(csv_path)

        if not nul_suspect:
            # Not a NUL/low-level byte issue → surface the real problem
            log.error("❌ COPY failed for %s (non-NUL error) – %s", csv_path.name, exc_raw)
            raise

        log.warning("COPY failed for %s (%s). Retrying with NUL-stripped temp file…",
                    csv_path.name, exc_raw)

        # ── 3) Disk-backed cleaning pass (constant memory) ─────────────────
        import tempfile
        with tempfile.NamedTemporaryFile("w+", encoding=file_encoding, delete=False) as tmp:
            tmp_path = Path(tmp.name)

            # Stream read → strip only NULs → stream write
            with csv_path.open("r", encoding=file_encoding, errors="replace", newline="") as src:
                for line in src:
                    tmp.write(line.replace("\x00", ""))

            tmp.flush()
            tmp.seek(0)

            try:
                cur.copy_expert(copy_sql, tmp)
                conn.commit()
                log.debug("✓ cleaned COPY (%s)  %s → %s", file_encoding, csv_path.name, target_table)
            except Exception as exc_clean:
                conn.rollback()
                log.error("❌ COPY still failing for %s after NUL-strip – %s",
                          csv_path.name, exc_clean)
                raise
            finally:
                # Best-effort cleanup
                try:
                    tmp_path.unlink(missing_ok=True)
                except Exception:
                    log.warning("⚠️ Could not delete temp file %s", tmp_path)

    finally:
        # ── 4) Always close resources ───────────────────────────────────────
        try:
            cur.close()
        finally:
            conn.close()
        log.debug("Closed cursor/connection for %s", csv_path.name)

# 6)  High-level loader · iterate CSV_DIR and COPY into matching temp tables

In [6]:
# ------------------------------------------------------------------
# 5 b)  High-level CSV loader  – iterates over CSV_DIR
# ------------------------------------------------------------------
from typing import Optional

def load_csvs_into_postgres(limit_files: Optional[int] = None) -> bool:
    """
    COPY every ``*.csv`` in ``CSV_DIR`` into its matching *temp_* table.

    Parameters
    ----------
    limit_files : int | None, default **None**
        • *None*  → ingest **all** CSVs present  
        • *N > 0* → stop after N files (great for quick tests / CI)

    Returns
    -------
    bool
        **True**  → *all* loads succeeded  
        **False** → at least one file failed (check the logs)
    """
    log.info(
        "🔄  CSV → Postgres loader started  •  limit=%s",
        limit_files if limit_files is not None else "∞"
    )

    # ------------------------------------------------------------------
    # 1) Collect CSV paths (alphabetical order for reproducibility)
    # ------------------------------------------------------------------
    csv_paths = sorted(p for p in CSV_DIR.iterdir() if p.suffix.lower() == ".csv")
    if limit_files is not None:
        csv_paths = csv_paths[:limit_files]

    if not csv_paths:
        log.warning("⚠️  No CSV files found in %s – nothing to load.", CSV_DIR)
        return False

    # ------------------------------------------------------------------
    # 2) Loop & COPY
    # ------------------------------------------------------------------
    all_ok   = True
    processed = 0

    for csv_path in csv_paths:
        # (a) Which layout/table does this file belong to?
        key = layout_key(csv_path.name)
        if not key:
            log.error("❌  Could not infer layout for %s – skipped.", csv_path.name)
            all_ok = False
            continue

        target_table = f"temp_{key}"

        # (b) COPY into Postgres via our low-level helper
        try:
            _copy_csv_to_pg(csv_path, target_table)
            processed += 1
        except Exception as err:            # already partially logged inside helper
            log.error("❌  Load failed for %s – %s", csv_path.name, err)
            all_ok = False

    # ------------------------------------------------------------------
    # 3) Summary
    # ------------------------------------------------------------------
    log.info(
        "🏁  Loader finished – %s file(s) processed – status: %s",
        processed,
        "SUCCESS" if all_ok else "PARTIAL FAILURE",
    )
    return all_ok


# 7)  Promote temp tables → final names (safer & noisier)

In [7]:
# ------------------------------------------------------------------
# 6 b)  Stand-alone promotion helper
# ------------------------------------------------------------------
def promote_temp_tables(status: Optional[bool] = None, **ctx) -> None:
    """
    Promote every *temp_* table to its final name — but only if the CSV loader
    succeeded.

    Works in two modes:
      • Airflow task: don't pass `status`; we read XCom from `load_csvs`.
      • Notebook / script: pass `status=True/False` explicitly.

    Safety rails
    ------------
    • Status gate: if status is falsy → abort promotion.
    • Best-effort: each table is attempted independently.
    • Idempotent: we DROP the final table (if exists) before RENAME.
    • Missing temp_*: logged as 'skipped'.
    """
    # 1) Resolve the loader status
    if status is None:
        # Airflow path: try to read from XCom, default to False if missing
        try:
            ti = ctx.get("ti")
            if ti is not None:
                status = bool(ti.xcom_pull(task_ids="load_csvs", key="status"))
            else:
                status = False
        except Exception:
            status = False

    if not status:
        log.error("🚫  CSV loader reported errors – temp-table promotion ABORTED.")
        return

    # 2) Promote table-by-table
    pg = Postgres()
    promoted = 0
    skipped  = 0
    errors   = 0

    for name in LAYOUTS.keys():
        temp_name = f"temp_{name}"
        log.info("🔄  Promoting  %s  →  %s", temp_name, name)

        try:
            # Try DROP final then RENAME temp->final.
            # If temp_* doesn't exist, the ALTER will throw and we'll log as 'skipped'.
            pg.execute_sql(f"DROP TABLE IF EXISTS {name};")
            pg.execute_sql(f"ALTER TABLE {temp_name} RENAME TO {name};")
            promoted += 1
            log.debug("✓  %s promoted successfully", name)

        except Exception as exc:
            msg = str(exc).lower()
            if "does not exist" in msg or "relation" in msg and "does not exist" in msg:
                # No temp_* for this layout this run — just skip noisily
                skipped += 1
                log.warning("↪️  Skipping %s – %s", name, exc)
            else:
                errors += 1
                log.error("❌  Failed promoting %s – %s", name, exc)

    # 3) Summary
    if errors == 0:
        log.info("🎉  Promotion finished – %s swapped, %s skipped, %s errors.",
                 promoted, skipped, errors)
    else:
        log.warning("⚠️  Promotion finished with %s error(s) – %s swapped, %s skipped.",
                    errors, promoted, skipped)


def _harden_final_schemas() -> None:
    """
    Convert TEXT-like date columns to proper DATE in final tables, cleaning common garbage:
    "", "0", "00000000", non-digits. Safe & idempotent:
      - Skips tables missing.
      - Skips columns already DATE.
      - For text-ish columns, casts using a robust CASE over col::text.
    """
    pg = Postgres()
    conn = pg.connect_postgres()
    cur  = conn.cursor()

    targets = {
        "estabelecimento": [
            "data_situacao_cadastral",
            "data_inicio_atividades",
            "data_situacao_especial",
        ],
        "simples": [
            "data_opcao_simples",
            "data_exclusao_simples",
            "data_opcao_mei",
            "data_exclusao_mei",
        ],
        "socios_original": [
            "data_entrada_sociedade",
        ],
    }

    def table_exists(tbl: str) -> bool:
        cur.execute("SELECT to_regclass('public.' || %s) IS NOT NULL;", (tbl,))
        return bool(cur.fetchone()[0])

    def column_type(tbl: str, col: str) -> str | None:
        cur.execute(
            """
            SELECT data_type
            FROM information_schema.columns
            WHERE table_schema='public' AND table_name=%s AND column_name=%s
            """,
            (tbl, col),
        )
        row = cur.fetchone()
        return row[0] if row else None  # e.g. 'text', 'character varying', 'date', ...

    # Expression that normalizes a TEXT-ish column into DATE.
    # Note: we always operate on col::text to support current type TEXT or DATE.
    def cast_expr(col: str) -> str:
        return f"""
        CASE
          WHEN NULLIF(regexp_replace(({col})::text, '\\D', '', 'g'), '') IS NULL
               OR NULLIF(regexp_replace(({col})::text, '\\D', '', 'g'), '') = '0'
            THEN NULL
          WHEN length(regexp_replace(({col})::text, '\\D', '', 'g')) = 8
            THEN to_date(regexp_replace(({col})::text, '\\D', '', 'g'), 'YYYYMMDD')
          WHEN ({col})::text ~ '^\\d{{4}}-\\d{{2}}-\\d{{2}}$'
            THEN (({col})::text)::date
          ELSE NULL
        END
        """

    altered_any = False

    for tbl, cols in targets.items():
        if not table_exists(tbl):
            log.info("↪️  Skip harden: table %s does not exist.", tbl)
            continue

        alters = []
        for col in cols:
            ctype = column_type(tbl, col)
            if ctype is None:
                log.info("↪️  Skip column: %s.%s not found.", tbl, col)
                continue
            if ctype.lower() == "date":
                log.debug("• %s.%s already DATE — skipping.", tbl, col)
                continue

            # Only non-DATE columns are altered
            alters.append(f"ALTER COLUMN {col} TYPE DATE USING {cast_expr(col)}")

        if alters:
            sql = f"ALTER TABLE {tbl} " + ", ".join(alters) + ";"
            cur.execute(sql)
            cur.execute(f"ANALYZE {tbl};")
            conn.commit()
            altered_any = True
            log.info("✅ Hardened %s (%d column(s)).", tbl, len(alters))
        else:
            log.info("↪️  Nothing to harden on %s.", tbl)

    if altered_any:
        log.info("🧱 Final schemas hardened (TEXT→DATE with cleanup).")
    else:
        log.info("ℹ️  No columns altered (already consistent).")

    try:
        cur.close()
    finally:
        conn.close()

    

# 8) Pipeline definition

In [8]:
# ▶ (extract → DDL → load → promote → harden)
from types import SimpleNamespace
import time

print("\n🧩 Running: extract_zip_to_csv → create_temp_tables → load_csvs → promote → harden")

t0 = time.perf_counter()

# 1) Extract ZIP → CSV
print("\n1️⃣  Extracting ZIPs…")
extract_zip_to_csv(limit=None)

# 2) (Re)create temp tables
print("\n2️⃣  Creating temp tables…")
create_temp_tables()

# 3) Load all CSVs into temp_* (us3 layout_key + COPY robust)
print("\n3️⃣  Loading CSVs into Postgres…")
status = load_csvs_into_postgres(limit_files=None)
print(f"   ↳ load status: {'SUCCESS' if status else 'PARTIAL FAILURE'}")

# 4) Promote temp_* → final
print("\n4️⃣  Promoting temp_* → final…")
fake_ctx = {"ti": SimpleNamespace(xcom_pull=lambda **kw: status)}
promote_temp_tables(status=status)

# 5) Harden final schemas (TEXT → DATE)
print("\n5️⃣  Hardening final schemas (TEXT→DATE)…")
_harden_final_schemas()

print(f"\n✅ Done in {time.perf_counter() - t0:.1f}s.")


21:58:42  INFO      nb-cnpj-ingest ➜ 🔧  ZIP ➜ CSV extraction started (archives=10)
21:58:42  INFO      nb-cnpj-ingest ➜ → [1/10]  Cnaes.zip
21:58:42  INFO      nb-cnpj-ingest ➜    ✔  done in 0.00s
21:58:42  INFO      nb-cnpj-ingest ➜ → [2/10]  Empresas0.zip
21:58:42  ERROR     nb-cnpj-ingest ➜ ❌  Bad ZIP Empresas0.zip – File is not a zip file
21:58:42  INFO      nb-cnpj-ingest ➜ → [3/10]  Estabelecimentos0.zip
21:58:42  INFO      nb-cnpj-ingest ➜    ✔  done in 0.00s
21:58:42  INFO      nb-cnpj-ingest ➜ → [4/10]  Motivos.zip
21:58:42  INFO      nb-cnpj-ingest ➜    ✔  done in 0.00s
21:58:42  INFO      nb-cnpj-ingest ➜ → [5/10]  Municipios.zip
21:58:42  INFO      nb-cnpj-ingest ➜    ✔  done in 0.00s
21:58:42  INFO      nb-cnpj-ingest ➜ → [6/10]  Naturezas.zip
21:58:42  INFO      nb-cnpj-ingest ➜    ✔  done in 0.00s
21:58:42  INFO      nb-cnpj-ingest ➜ → [7/10]  Paises.zip
21:58:42  INFO      nb-cnpj-ingest ➜    ✔  done in 0.00s
21:58:42  INFO      nb-cnpj-ingest ➜ → [8/10]  Qualificacoes.


🧩 Running: extract_zip_to_csv → create_temp_tables → load_csvs → promote → harden

1️⃣  Extracting ZIPs…

2️⃣  Creating temp tables…


DETAIL:  The database was created using collation version 2.36, but the operating system provides version 2.41.
HINT:  Rebuild all objects in this database that use the default collation and run ALTER DATABASE airflow REFRESH COLLATION VERSION, or build PostgreSQL with the right library version.
DETAIL:  The database was created using collation version 2.36, but the operating system provides version 2.41.
HINT:  Rebuild all objects in this database that use the default collation and run ALTER DATABASE airflow REFRESH COLLATION VERSION, or build PostgreSQL with the right library version.
DETAIL:  The database was created using collation version 2.36, but the operating system provides version 2.41.
HINT:  Rebuild all objects in this database that use the default collation and run ALTER DATABASE airflow REFRESH COLLATION VERSION, or build PostgreSQL with the right library version.
DETAIL:  The database was created using collation version 2.36, but the operating system provides version 2.4


3️⃣  Loading CSVs into Postgres…


DETAIL:  The database was created using collation version 2.36, but the operating system provides version 2.41.
HINT:  Rebuild all objects in this database that use the default collation and run ALTER DATABASE airflow REFRESH COLLATION VERSION, or build PostgreSQL with the right library version.
DETAIL:  The database was created using collation version 2.36, but the operating system provides version 2.41.
HINT:  Rebuild all objects in this database that use the default collation and run ALTER DATABASE airflow REFRESH COLLATION VERSION, or build PostgreSQL with the right library version.
DETAIL:  The database was created using collation version 2.36, but the operating system provides version 2.41.
HINT:  Rebuild all objects in this database that use the default collation and run ALTER DATABASE airflow REFRESH COLLATION VERSION, or build PostgreSQL with the right library version.
DETAIL:  The database was created using collation version 2.36, but the operating system provides version 2.4

   ↳ load status: PARTIAL FAILURE

4️⃣  Promoting temp_* → final…

5️⃣  Hardening final schemas (TEXT→DATE)…

✅ Done in 243.9s.


In [9]:
# %% Load only Estabelecimentos0.zip → temp_estabelecimento (binary COPY, robust logs)

from pathlib import Path
import zipfile, io, time, logging
from postgre_connector import Postgres  # usa suas credenciais .env via helper

# ── repo/datasets detection (same logic used before) ─────────────────
REPO_ROOT = Path.cwd().resolve()
while REPO_ROOT != REPO_ROOT.parent:
    if (REPO_ROOT / "airflow" / "dags").is_dir():
        break
    REPO_ROOT = REPO_ROOT.parent

BASE_DIR = REPO_ROOT / "airflow" / "datasets"
ZIP_DIR  = BASE_DIR / "public-zips"
CSV_DIR  = BASE_DIR / "public-data"
ZIP_DIR.mkdir(parents=True, exist_ok=True)
CSV_DIR.mkdir(parents=True, exist_ok=True)

# ── logging ─────────────────────────────────────────────────────────
log = logging.getLogger("estab0-loader")
if not log.handlers:
    h = logging.StreamHandler()
    h.setFormatter(logging.Formatter("%(asctime)s  %(levelname)-8s  %(message)s", "%H:%M:%S"))
    log.addHandler(h)
log.setLevel(logging.INFO)

# ── helpers ─────────────────────────────────────────────────────────
def extract_target_csv_from_zip(zpath: Path, out_dir: Path) -> Path:
    """
    Extract the *largest* CSV inside the ZIP (typical Estabelecimentos0 has a single big CSV).
    Returns the extracted path.
    """
    with zipfile.ZipFile(zpath, "r") as z:
        # list only CSV-like entries
        members = [m for m in z.namelist() if m.lower().endswith(".csv") or "estabele" in m.lower()]
        if not members:
            raise FileNotFoundError("No CSV-like members found inside the ZIP.")
        # pick the largest by uncompressed size
        info_by_name = {i.filename: i for i in z.infolist()}
        target_name = max(members, key=lambda n: info_by_name[n].file_size)
        tgt = out_dir / f"{zpath.stem}__{Path(target_name).name if Path(target_name).suffix else Path(target_name).name + '.csv'}"
        if not tgt.exists():
            log.info("Extracting %s → %s", target_name, tgt.name)
            with z.open(target_name) as src, open(tgt, "wb") as dst:
                # stream copy
                while True:
                    chunk = src.read(1 << 20)  # 1 MiB
                    if not chunk:
                        break
                    dst.write(chunk)
        else:
            log.info("Using existing extracted file: %s", tgt.name)
        return tgt

def copy_csv_binary(csv_path: Path, table: str = "temp_estabelecimento") -> None:
    """
    COPY FROM STDIN robust:
      - client_encoding LATIN1
      - NULL '' (string vazia → NULL)
      - streaming com strip de NUL sem carregar tudo na memória
      - evita reinterpretação como UTF-8
    Pré-requisito: staging sem FK e datas como TEXT.
    """
    from postgre_connector import Postgres
    import io

    class TextStreamingCleaner:
        """Gerador de texto: lê bytes LATIN1 em blocos, remove NUL e entrega str."""
        def __init__(self, path: Path, chunk_bytes: int = 1 << 20):
            self.path = path
            self.chunk_bytes = chunk_bytes
            self._fh = None
        def __enter__(self):
            self._fh = open(self.path, "rb")
            return self
        def __exit__(self, *exc):
            try:
                if self._fh: self._fh.close()
            except Exception:
                pass
        def read(self, n: int = -1) -> str:
            if self._fh is None:
                return ""
            # lê um bloco bruto e converte para texto LATIN1, removendo NUL
            raw = self._fh.read(self.chunk_bytes if n < 0 else n)
            if not raw:
                return ""
            return raw.replace(b"\x00", b"").decode("latin-1", errors="replace")

    pg = Postgres()
    conn = pg.connect_postgres()
    cur  = conn.cursor()
    try:
        # Importantíssimo: diga ao servidor que STDIN virá em LATIN1
        cur.execute("SET client_encoding TO 'LATIN1';")
        cur.execute("SET statement_timeout = 0;")
        cur.execute("SET synchronous_commit = off;")

        copy_sql = f"""
        COPY {table}
        FROM STDIN WITH (
          FORMAT csv,
          DELIMITER ';',
          NULL '',
          HEADER false
        );
        """

        # Caminho 1: streaming textual (já decodificado em LATIN1)
        t0 = time.time()
        with TextStreamingCleaner(csv_path) as reader:
            cur.copy_expert(copy_sql, reader)  # espera str → respeita client_encoding
        conn.commit()
        log.info("COPY done (stream LATIN1 + NUL-strip) in %.1fs", time.time() - t0)

        # Conferência rápida
        cur.execute("SELECT COUNT(*) FROM temp_estabelecimento;")
        rows = cur.fetchone()[0]
        log.info("Row count now in temp_estabelecimento: %s", rows)

    finally:
        try:
            cur.close(); conn.close()
        except Exception:
            pass

       

# ── main: pick Estabelecimentos0.zip only, extract, COPY ────────────
zfile = ZIP_DIR / "Estabelecimentos0.zip"
if not zfile.exists():
    raise FileNotFoundError(f"{zfile} not found. Make sure Estabelecimentos0.zip is in {ZIP_DIR}")

if not zipfile.is_zipfile(zfile):
    raise RuntimeError(f"{zfile} is not a valid ZIP (might be a partial/HTML download).")

log.info("Using ZIP: %s (%.2f GB)", zfile.name, zfile.stat().st_size / (1<<30))
csv_path = extract_target_csv_from_zip(zfile, CSV_DIR)
log.info("Target CSV: %s (%.2f GB)", csv_path.name, csv_path.stat().st_size / (1<<30))

# heads up: Receita CSV NÃO tem header; se o preview mostrar nomes de coluna, mude HEADER true
try:
    # quick preview
    with csv_path.open("rb") as fh:
        head = fh.read(4096).decode("latin-1", errors="replace").splitlines()[:2]
    log.info("Preview L1: %s", head[0] if head else "<empty>")
    log.info("Preview L2: %s", head[1] if len(head) > 1 else "<none>")
except Exception:
    pass

# execute COPY
copy_csv_binary(csv_path, "temp_estabelecimento")


22:02:46  INFO      Using ZIP: Estabelecimentos0.zip (1.73 GB)
22:02:46  INFO      estab0-loader ➜ Using ZIP: Estabelecimentos0.zip (1.73 GB)
22:02:46  INFO      Extracting K3241.K03200Y0.D51011.ESTABELE → Estabelecimentos0__K3241.K03200Y0.D51011.ESTABELE
22:02:46  INFO      estab0-loader ➜ Extracting K3241.K03200Y0.D51011.ESTABELE → Estabelecimentos0__K3241.K03200Y0.D51011.ESTABELE


22:02:59  INFO      Target CSV: Estabelecimentos0__K3241.K03200Y0.D51011.ESTABELE (5.51 GB)
22:02:59  INFO      estab0-loader ➜ Target CSV: Estabelecimentos0__K3241.K03200Y0.D51011.ESTABELE (5.51 GB)
22:03:00  INFO      Preview L1: "11779918";"0001";"05";"1";"";"04";"20210310";"63";"";"";"20100312";"5611204";"5611203";"RUA";"ANA NERI";"73";"";"CAMBUI";"13024500";"SP";"6291";"19";"91174491";"";"";"";"";"kaconori@ig.com.br";"";""
22:03:00  INFO      estab0-loader ➜ Preview L1: "11779918";"0001";"05";"1";"";"04";"20210310";"63";"";"";"20100312";"5611204";"5611203";"RUA";"ANA NERI";"73";"";"CAMBUI";"13024500";"SP";"6291";"19";"91174491";"";"";"";"";"kaconori@ig.com.br";"";""
22:03:00  INFO      Preview L2: "11779934";"0001";"90";"1";"JOIAS VIP";"08";"20120927";"01";"";"";"20100325";"4783101";"4783102,4618499";"RUA";"PRESIDENTE FARIA";"282";"ANDAR 3                   SALA  303";"CENTRO";"80020290";"PR";"7535";"41";"31112626";"";"";"41";"31112626";"";"";""
22:03:00  INFO      estab0-loader ➜